# funções de perda

In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

# instanciando a GPU

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


# CLASSIFICAÇÃO

# importando o dataset

In [3]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine. target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)


(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']


# criando a rede neural para classificação

In [4]:
## definindo os parametros
input_size  = data.shape[1]  ## quantidade de X do data set
hidden_size = 32
out_size = len(wine.target_names) ## quantidade de Y

# fazendo a class nn.Module
class WineClassifier(nn.Module):
    
    def __init__(self, input_size, hidden_size, out_size):
        super(WineClassifier, self).__init__()
        
        self.hidden  = nn.Linear(input_size, hidden_size)
        self.relu    = nn.ReLU()
        self.out     = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()
        
    def forward(self, X):
        
        feature = self.relu(self.hidden(X))
        output = self.softmax(self.out(feature))
        
        
        return output
    

## jogando os dados na GPU
net = WineClassifier(input_size, hidden_size, out_size).to(device)  ## cast na GPU
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


# instanciando a função de perda em função da classificacao

## utilizando entropya cruzada

In [5]:
criterion = nn.CrossEntropyLoss().to(device) ##cast na GPU

# transformando os dados em tensores

In [17]:
Xtns = torch.from_numpy(data).float() ##como os dados sao array nunmpy, fou usado o comando from_numpy e transformando em float 32
Ytns = torch.from_numpy(target).long() ## transformar int 32 para int 64


Xtns = Xtns.to(device) #cast na GPU
Ytns = Ytns.to(device) #cast na GPU

print(Xtns.dtype, Ytns.dtype)

torch.float32 torch.int64


In [13]:
pred = net(Xtns)

<ipython-input-4-a84a0e39d1c4>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(self.out(feature))


In [14]:
print(pred.shape, Ytns.shape)

torch.Size([178, 3]) torch.Size([178])


In [15]:
print(pred[0].data, Ytns[0].data)

tensor([0., 1., 0.], device='cuda:0') tensor(0, device='cuda:0')


In [18]:
##### só funciona se o Y estiver em int64

loss = criterion(pred, Ytns)
print(loss)

tensor(1.1526, device='cuda:0', grad_fn=<NllLossBackward0>)


# REGRESSÃO

## importando o dataset

In [25]:
from sklearn import datasets

diabetes = datasets.load_diabetes()

data = diabetes.data
target = diabetes.target

print(data.shape, target.shape)
print(data[14])
print(target[14])

(442, 10) (442,)
[ 4.53409833e-02 -4.46416365e-02 -2.56065715e-02 -1.25563519e-02
  1.76943802e-02 -6.12835791e-05  8.17748397e-02 -3.94933829e-02
 -3.19914449e-02 -7.56356220e-02]
118.0


# Criando a rede neural para Regressão

In [33]:
## definindo os parametros
input_size  = data.shape[1]  ## quantidade de X do data set
hidden_size = 32
out_size = 1 ## quantidade de Y (tem apenas 1 variável) // #progressão da diabetes

# fazendo a class nn.Module
class WineClassifier(nn.Module):
    
    def __init__(self, input_size, hidden_size, out_size):
        super(WineClassifier, self).__init__()
        
        self.hidden  = nn.Linear(input_size, hidden_size)
        self.relu    = nn.ReLU()
        self.out     = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, X):
        
        feature = self.relu(self.hidden(X))
        output = self.softmax(self.out(feature))
        
        
        return output
    

## jogando os dados na GPU
net = WineClassifier(input_size, hidden_size, out_size).to(device)  ## cast na GPU
print(net)

WineClassifier(
  (hidden): Linear(in_features=10, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=1, bias=True)
  (softmax): Softmax(dim=-1)
)


# Definindo a função de perda

## Utilizando a função MSELoss = mean squared erros

### importante: quandor for passar no loss, tanto o Ytns quanto o pred tem que ter a mesma dimensão se não dá ERRO

In [34]:
criterion = nn.MSELoss().to(device)  #subir na GPU

# cast da GPU

Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

print(Xtns.shape, Ytns.shape)
print(Xtns.dtype, Ytns.dtype)

torch.Size([442, 10]) torch.Size([442])
torch.float32 torch.float32


In [35]:
pred = net(Xtns)
print(pred.shape)

loss = criterion(pred.squeeze(), Ytns)   #função squeeze/unsqueeze modifica os dados, no caso, o pred foi modificado para ter o mesmo tamanho do ytns
print(loss.data)

torch.Size([442, 1])
tensor(28771.2168, device='cuda:0')


## utilizando outra função de critério, L1loss => Creates a criterion that measures the mean absolute error (MAE) between each element

In [38]:
# utilizando outra função de criterio

criterion = nn.L1Loss().to(device)

Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

pred = net(Xtns)
print(pred.shape)

loss = criterion(pred.squeeze(), Ytns)   #função squeeze/unsqueeze modifica os dados, no caso, o pred foi modificado para ter o mesmo tamanho do ytns
print(loss.data)

torch.Size([442, 1])
tensor(151.1335, device='cuda:0')
